<a href="https://colab.research.google.com/github/Erikaklim/image-classification-evaluation-with-pretrained-model/blob/main/lab1final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openimages

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.3 MB/s eta 0:00:00


In [ ]:
import os
from openimages.download import download_dataset
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from openimages.download import download_dataset

In [ ]:
data_dir = "data"
number_of_samples = 100
dic = {"Banana": 954, "Bicycle": 444, "Table": 532}
classes = list(dic.keys())
indices = list(dic.values())
T = [0.5, 0.8, 0.8]

In [ ]:
if not os.path.exists(data_dir):
  os.makedirs(data_dir)

download_dataset(data_dir, classes, limit=number_of_samples)

100%|██████████| 100/100 [00:03<00:00, 28.93it/s]


{'banana': {'images_dir': 'data/banana/images'},
 'bicycle': {'images_dir': 'data/bicycle/images'},
 'table': {'images_dir': 'data/table/images'}}

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory='data',
    labels='inferred',
    label_mode='categorical',
    image_size=(384, 384),
    batch_size=32,
    shuffle=False,
)

Found 300 files belonging to 3 classes.


In [ ]:
train_ds.class_names

['banana', 'bicycle', 'table']

In [ ]:
GT=[]
for i in train_ds:
    GT.extend(i[1].numpy())
GT=np.array(GT)


In [ ]:
GT[:4]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = tf.keras.applications.efficientnet_v2.EfficientNetV2S(weights='imagenet', classifier_activation=None)
p=model.predict(train_ds)
p_sigmoid = tf.nn.sigmoid(p).numpy()

10/10 [==============================] - 157s 15s/step


In [ ]:
filtered_p = p_sigmoid[:, indices]
filtered_p[:10]

array([[0.99437565, 0.44967565, 0.5905933 ],
       [0.9998879 , 0.6503457 , 0.29258046],
       [0.99722445, 0.490421  , 0.92366815],
       [0.99988407, 0.6045027 , 0.36962274],
       [0.99889755, 0.46561706, 0.8693796 ],
       [0.9997267 , 0.51479906, 0.45745766],
       [0.9999406 , 0.61279535, 0.34205896],
       [0.9999191 , 0.6093657 , 0.35120243],
       [0.98799896, 0.52766246, 0.3410908 ],
       [0.9991727 , 0.7398686 , 0.35097173]], dtype=float32)

In [ ]:
filtered_p[:10] > T

array([[ True, False, False],
       [ True, False, False],
       [ True, False,  True],
       [ True, False, False],
       [ True, False,  True],
       [ True, False, False],
       [ True, False, False],
       [ True, False, False],
       [ True, False, False],
       [ True, False, False]])

In [ ]:
predictions = np.int16(filtered_p > T)
predictions[:10]

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]], dtype=int16)

In [ ]:
greatest_values = np.argmax(filtered_p, axis=1)
greatest_values[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
results = [
    classes[i]
    for i in greatest_values
    for item in filtered_p
    if(item[i] > T[i])
]
results[:10]

['Banana',
 'Banana',
 'Banana',
 'Banana',
 'Banana',
 'Banana',
 'Banana',
 'Banana',
 'Banana',
 'Banana']

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(GT, predictions)


0.5266666666666666

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(GT, predictions))

              precision    recall  f1-score   support

           0       0.45      1.00      0.62       100
           1       0.93      0.97      0.95       100
           2       0.92      0.83      0.87       100

   micro avg       0.67      0.93      0.78       300
   macro avg       0.77      0.93      0.81       300
weighted avg       0.77      0.93      0.81       300
 samples avg       0.73      0.93      0.80       300

